# A* Algo

In [61]:
import numpy as np
n = 10
matrix = []
block_unblock_matrix = []

np_array = np.random.randint(0, 100, n*n).reshape(n, n)
matrix = np.asmatrix(np_array)

matrix

matrix([[77,  5, 75, 96, 91, 57,  4, 99, 78, 57],
        [32,  4,  9,  5, 96,  7,  2, 52, 49,  5],
        [86, 73, 78,  4, 28,  7, 99, 30, 51, 30],
        [12, 74, 54, 17, 93, 44,  9, 48, 38, 25],
        [34, 30, 70, 60, 54, 84, 71, 12, 66, 44],
        [15, 57, 78, 52, 91, 57, 83, 54, 97, 83],
        [41,  4, 95, 53, 46, 98, 54,  0, 98, 27],
        [20, 31, 71, 39,  0, 95, 61, 29, 45, 34],
        [62, 69, 63, 22, 85, 43,  4, 80, 42, 66],
        [79, 10, 90,  3, 31, 80, 40,  3, 45, 37]])

In [62]:
np_array = np.random.randint(0, 2, n*n).reshape(n, n)
block_unblock_matrix = np.asmatrix(np_array)

block_unblock_matrix

matrix([[0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
        [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 1, 1, 1, 0],
        [1, 0, 0, 1, 0, 1, 0, 1, 1, 1],
        [1, 1, 1, 0, 0, 1, 1, 1, 0, 0],
        [0, 0, 1, 0, 0, 1, 1, 0, 1, 1],
        [1, 1, 0, 0, 1, 1, 0, 1, 0, 0],
        [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
        [1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 0, 0, 1, 1]])

In [63]:
# [[1 for j in range(n)] for i in range(n)]

# hofn_matrix = list(np.array(np.arange(100).reshape((10,10))))
# # np.fliplr(hofn_matrix)

def compute_hofn(n):
    k = 0
    hofn = []
    one_row = []
    for j in range(n):
        one_row.append(k)
        k += 1

    hofn.append(one_row)

    for i in range(n-1):
        one_row = [x+1 for x in one_row]
        hofn.append(one_row)

    hofn_matrix = np.array([np.array(xi) for xi in hofn])
    hofn_matrix = np.flip(hofn_matrix)
    return hofn_matrix

g(n) matrix

In [64]:
def compute_gofn(n):
    k = 0
    gofn = []
    one_row = []
    for j in range(n):
        one_row.append(k)
        k += 1
    gofn.append(one_row)
    for i in range(n-1):
        one_row = [x+1 for x in one_row]
        gofn.append(one_row)

    gofn_matrix = np.array([np.array(xi) for xi in gofn])
    return gofn_matrix

In [65]:
def get_gofn(gofn_matrix, row, col):
    return gofn_matrix[row][col]

In [66]:
def get_hofn(hofn_matrix, row, col):
    return hofn_matrix[row][col]

In [72]:
def get_children(matrix, row, column, n):
    children = []
    if row == 0 and column == 0:
        children.append((row+1,column))
        children.append((row, column+1))
    elif row == n-1 and column == n-1:
        print("goal node doesn't have children")
    elif row == n-1:
        children.append((row, column+1))
    elif column == n-1:
        children.append((row+1, column))
    else:
        print("Invalid node")
    return children 

In [73]:
def sort(open_list, gofn_matrix, hofn_matrix):
    sort_dict = {}
    for node in open_list:
        row = node[0]
        col = node[1]
        gofn = gofn_matrix.item(start)
        hofn = hofn_matrix[row][col]
        fofn = gofn + hofn
        sort_dict[fofn] = node
    sort_dict = dict(sorted(sort_dict.items(), key=lambda item: item[1]))
    return sort_dict

In [76]:
visited_list = []
open_list = []

start = (0,0)
goal = (n-1, n-1)

open_list.append(start)
print(open_list)
gofn_matrix = compute_gofn(n)
hofn_matrix = compute_hofn(n)
while open_list:
    current_node = open_list.pop()
    if block_unblock_matrix.item(current_node) == 0:
        visited_list.append(current_node)
        children = get_children(block_unblock_matrix, current_node[0], current_node[1], n)
        print("children", children)
        open_list.extend(children)
        sort_dict = sort(open_list, gofn_matrix, hofn_matrix)
        print("open_list", open_list)
        print("\n", sort_dict)
#         break

[(0, 0)]
children [(1, 0), (0, 1)]
open_list [(1, 0), (0, 1)]

 {17: (0, 1)}
Invalid node
children []
open_list [(1, 0)]

 {17: (1, 0)}
